In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder \
.master("local[*]") \
.appName("test") \
.getOrCreate()

In [ ]:
df_green = spark.read.parquet('data/raw/green')

In [ ]:
df_green.createOrReplaceTempView('green')

In [ ]:
df_green_revenue =  spark.sql(
    """
    SELECT  
         date_trunc('hour', lpep_pickup_datetime) AS hour,
         PULocationID AS zone,
         

         SUM(total_amount) AS amount,
         COUNT(1) AS number_records
    FROM
        green
    WHERE
         lpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1,2
    ORDER BY
        1,2
    """
) 

In [ ]:
df_green_revenue.write.parquet('data/raw/report/revenue/green', mode='overwrite')

In [ ]:
spark

In [ ]:
df_yellow = spark.read.parquet('data/raw/yellow')

In [ ]:
df_yellow.show()

In [ ]:
df_yellow.createOrReplaceTempView('yellow')

In [ ]:
df_yellow_revenue = spark.sql(

 """
    SELECT  
         date_trunc('hour', tpep_pickup_datetime) AS hour,
         PULocationID AS zone,
         

         SUM(total_amount) AS amount,
         COUNT(1) AS number_records
    FROM
        yellow
    WHERE
         tpep_pickup_datetime >= '2020-01-01 00:00:00'
    GROUP BY 
        1,2
    ORDER BY
        1,2
    """

)

In [ ]:
df_yellow_revenue.show()

In [ ]:
df_yellow_revenue.repartition(20).write.parquet('data/raw/report/yellow', mode='overwrite')

In [ ]:
df_green_revenue_tmp = df_green_revenue \
.withColumnRenamed('amount','green_amount')\
.withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
.withColumnRenamed('amount','yellow_amount') \
.withColumnRenamed('number_records', 'yellow_number_records')

In [ ]:
#f_yellow_revenue, df_yellow_revenue

In [ ]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [ ]:
df_join.show()

In [ ]:
df_join.write.parquet('data/raw/report/revenue/total')

In [ ]:
df_join = spark.read.parquet('data/raw/report/revenue/total')

In [ ]:
df_join

In [ ]:
df_zones = spark.read.parquet('zones/')